# 0. GPT From Scratch

이 노트북은 GPT 논문을 기반으로 간단한 클론코딩을 통해 세익스피어풍의 작품을 생성해내는 언어 모델을 구현한 것입니다.

1. dataset:  tiny shakespere dataset 
2. tokenization : character-level 

언어 모델의 학습 방법론과 트랜스포머 구조에 대한 아주 기초적인 이해를 보유하고 있다고 가정하고 작성을 진행하겠습니다.

# 1. Dataset Preperation & Analysis

In [15]:
with open('./datas/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [41]:
# 데이터셋에 대한 여러 가지를 확인해봅니다. 
print("데이터셋의 총 길이:", len(text))
print("데이터셋의 첫 1000자:\n==============================\n" + text[:1000] + "==============================" )

데이터셋의 총 길이: 1115394
데이터셋의 첫 1000자:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for 

# 2. Tokenizing & Encoding Text - chararcter level

- 간단하게 character level로 토큰화 및 인코딩하기로 했습니다. 
- 구현이 간편하지만 인코딩된 토큰이 의미를 담아내지 못하므로
- 모델 성능은 상당히 낮을 것으로 예상됩니다. 

In [44]:
# Dataset의 모든 character 추출 및 등장 빈도만큼 오름차순 정렬
# character-level이므로 vocab은 등장한 모든 문자의 모음이 됩니다. 
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("vocab:", ''.join(chars))
print("vocab size:", vocab_size)

vocab: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


In [45]:
# 문자 하나 당 번호 하나를 매깁니다. 
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

# 문자열과 벡터를 오가는 인코딩/디코딩 함수를 만들어줍니다. 
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print("========== 예시 ==========")
print(encode("hi there!"))
print(decode(encode("What's your name?")))

========== 예시 ==========
[46, 47, 1, 58, 46, 43, 56, 43, 2]
What's your name?


In [ ]:
# 전체 데이터셋 인코딩

# 위 인코더로 text를 인코딩한 뒤 torch tensor 형태로 전체 데이터를 준비해둡니다. 
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print("data의 shape:", data.shape)
print("data의 dtype:", data.dtype)
# print(data[:1000])

data의 shape: torch.Size([1115394])
data의 dtype: torch.int64


# 3. Train / Validation / Test Split

In [49]:
# train과 test data를 분리한다.
# 비율은 약 10%로 정함.
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

이 데이터들을 "batch * context length" shape의 tensor 단위로 묶어낸다. 

In [54]:
# 모델의 context window size를 정한다. 
context_length = 8
print(train_data[:context_length + 1])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


In [55]:
# 예시
x = train_data[:context_length]
y = train_data[1: context_length + 1]
for i in range(context_length):
    context = x[:i + 1]
    target = y[i]
    print(f'input이 {context}일 때의 target: {target}')

input이 tensor([18])일 때의 target: 47
input이 tensor([18, 47])일 때의 target: 56
input이 tensor([18, 47, 56])일 때의 target: 57
input이 tensor([18, 47, 56, 57])일 때의 target: 58
input이 tensor([18, 47, 56, 57, 58])일 때의 target: 1
input이 tensor([18, 47, 56, 57, 58,  1])일 때의 target: 15
input이 tensor([18, 47, 56, 57, 58,  1, 15])일 때의 target: 47
input이 tensor([18, 47, 56, 57, 58,  1, 15, 47])일 때의 target: 58


In [53]:
# 모델의 batch size를 정한다. 
torch.manual_seed(1337)
batch_size = 4
context_length = 8

def get_batch(split):
    '''
    get_batch: 

    split: 어떤 데이터에 대한 batch를 불러올 건지 결정. 
    'train'인 경우 train_data에 대해서, 아닌 경우 val_data에 대해서 가져옴.
    '''
    # 데이터 불러오기
    data = train_data if split == 'train' else val_data 

    # batch size 개수의 랜덤한 offset 생성. 
    ix = torch.randint(len(data) - context_length, (batch_size,)) 

    # 랜덤 생성한 시작지점 ix에 대한 x, y 가져오기
    x = torch.stack([data[i : i + context_length] for i in ix])
    y = torch.stack([data[i + 1 : i + context_length + 1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('입력: ')
print(xb.shape)
print(xb)
print('target: ')
print(yb.shape)
print(yb)

입력: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
